# Imports

In [1]:
!pip install transformers
!pip install sentence_transformers
!pip install datasets
!pip install sentencepiece
!pip install scikit-learn-extra
!pip install wikipedia
!pip install eli5

     |████████████████████████████████| 2.3MB 14.5MB/s 
     |████████████████████████████████| 3.3MB 49.6MB/s 
     |████████████████████████████████| 901kB 42.4MB/s 
     |████████████████████████████████| 81kB 6.4MB/s 
     |████████████████████████████████| 1.2MB 19.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=48858d050e2209366e3e558d3618e15e3db1823d4b99d6d5aeb99ff6515ef9ce
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers
     |████████████████████████████████| 225kB 13.4MB/s 
     |████████████████████████████████| 245kB 19.8MB/s 
     |████████████████████████████████| 112kB 25.4MB/s 
     |████████████████████████████████| 1.7MB 13.2MB/s 
     |████████████████████████████████| 22.3MB 1.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Succe

In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import argparse
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import os
import random
import shutil
import wikipedia
import time
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from tqdm import tqdm
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
from PIL import Image
import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import sys
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.gaussian_process.kernels import RBF
import torch
from sklearn.metrics.pairwise import rbf_kernel

from scipy.stats import multivariate_normal
import  scipy.stats as st
from matplotlib import cm
import torch.optim as optim
from __future__ import print_function
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW, AutoModel
import nltk
from torch.utils.data import DataLoader
import pickle
import spacy
from gensim import corpora, models, similarities
import gensim
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.cluster import KMeans
import pickle
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import matplotlib
from sklearn.metrics.pairwise import rbf_kernel
import copy
import eli5
from eli5.lime import TextExplainer
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

cuda:0
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# HotpotQA


In [4]:
DISTRACTORS_PARS_LEN = 0

In [5]:
from datasets import load_dataset

dataset = load_dataset("hotpot_qa", 'distractor')
train_dataset = dataset['train']
validation_dataset = dataset['validation']

Dataset hotpot_qa downloaded and prepared to /root/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5. Subsequent calls will reuse this data.


In [6]:
def get_token_lenght(context, question):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    return len(inputs['input_ids'][0])

def remove_distractors(example, n_distractors = 0):
    all_pars = example['context']['title']
    gold_pars = list(set(example['supporting_facts']['title']))
    distractor_pars = list(set(all_pars) - set(gold_pars))
    # get indices to keep from disractors
    if len(distractor_pars) == 0 or n_distractors == 0:
        distract_indices = []
    else:
        distract_indices = random.sample(range(len(distractor_pars)), n_distractors)
    distractor_pars = [distractor_pars[idx] for idx in distract_indices]
    keep_pars = gold_pars + distractor_pars
    keep_pars_indices = [all_pars.index(keep_par) for keep_par in keep_pars]
    example['context']['title'] = [example['context']['title'][idx] for idx in keep_pars_indices]
    example['context']['sentences'] = [example['context']['sentences'][idx] for idx in keep_pars_indices]
    
    sentences_par = ["".join(example['context']['sentences'][idx]) for idx in range(len(example['context']['sentences']))]
    example['intros'] = " ".join([example['context']['sentences'][idx][0] for idx in range(len(example['context']['sentences']))])
    example['passage'] = " #$ ".join(sentences_par)
    return example

def remove_distractors_remove_key_sent(example, n_distractors = 0):
    all_pars = example['context']['title']
    gold_pars = list(set(example['supporting_facts']['title']))
    distractor_pars = list(set(all_pars) - set(gold_pars))
    # get indices to keep from disractors
    if len(distractor_pars) == 0 or n_distractors == 0:
        distract_indices = []
    else:
        distract_indices = random.sample(range(len(distractor_pars)), n_distractors)
    distractor_pars = [distractor_pars[idx] for idx in distract_indices]
    keep_pars = gold_pars + distractor_pars
    keep_pars_indices = [all_pars.index(keep_par) for keep_par in keep_pars]
    example['context']['title'] = [example['context']['title'][idx] for idx in keep_pars_indices]
    example['context']['sentences'] = [example['context']['sentences'][idx] for idx in keep_pars_indices]

    # removed all distractors, now we remove a key sentence
    rand_sup_idx =random.sample(list(range(len(example['supporting_facts']['sent_id']))), 1)[0] 
    title_to_index = {}
    for i in range(len(example['context']['title'])):
        title_to_index[example['context']['title'][i]] = i
    par_to_remove = title_to_index[example['supporting_facts']['title'][rand_sup_idx]]
    sent_to_remove = example['supporting_facts']['sent_id'][rand_sup_idx]
    sentencess = copy.deepcopy(example['context']['sentences'])
    able_to_remove = False
    times_tried = 0
    while not able_to_remove:
        if times_tried >4:
            print("giving up")
            break
        try:
            sent = sentencess[par_to_remove][sent_to_remove]
            if example['answer'] in sent:
                raise Exception('error')
            #sentencess[par_to_remove].pop(sent_to_remove)
            sentencess[par_to_remove][sent_to_remove] = " [...] . "
            able_to_remove = True
        except:
            times_tried += 1
            print("failed trying again")
            rand_sup_idx =random.sample(list(range(len(example['supporting_facts']['sent_id']))), 1)[0] 
            title_to_index = {}
            for i in range(len(example['context']['title'])):
                title_to_index[example['context']['title'][i]] = i
            par_to_remove = title_to_index[example['supporting_facts']['title'][rand_sup_idx]]
            sent_to_remove = example['supporting_facts']['sent_id'][rand_sup_idx]
            sentencess = copy.deepcopy(example['context']['sentences'])

    example['intros'] = " ".join([sentencess[idx][0] for idx in range(len(sentencess)) if len(sentencess[idx])>0])

    sentences_par = ["".join(sentencess[idx]) for idx in range(len(sentencess))]

    example['passage'] = " #$ ".join(sentences_par)
    return example




In [7]:
train_dataset_hum = train_dataset.map(remove_distractors_remove_key_sent)
validation_dataset_hum = validation_dataset.map(remove_distractors_remove_key_sent)
train_dataset = train_dataset_hum#train_dataset.map(remove_distractors_remove_key_sent)
validation_dataset = validation_dataset_hum#validation_dataset.map(remove_distractors_remove_key_sent)

Streaming output truncated to the last 5000 lines.
failed trying again
failed trying again
failed trying again
giving up
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
giving up
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
giving up
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again

Streaming output truncated to the last 5000 lines.
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
giving up
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
giving up
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
giving up
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again
failed trying again

In [8]:
# Free form vs yes/no
all_indices = set(list(range(len(validation_dataset))))
yes_no_indices = set([i  for i in range(len(validation_dataset)) if validation_dataset[i]['answer'] in ["yes","no"] ])
free_indices_val = all_indices - yes_no_indices
all_indices = set(list(range(len(train_dataset))))
yes_no_indices_train = set([i  for i in range(len(train_dataset)) if (train_dataset[i]['answer'] in ["yes","no"]) or (train_dataset[i]['level'] in ['easy','medium']) ])
free_indices_train = all_indices - yes_no_indices_train
print(len(free_indices_val))
print(len(free_indices_train))


6947
14631


In [9]:
hard_dataset = [train_dataset[i] for i in free_indices_train ]
hard_dataset += [validation_dataset[i] for i in free_indices_val]
dataset_indices =  set(list(range(len(hard_dataset))))
VAL_START = len([train_dataset[i] for i in free_indices_train ])
len(hard_dataset)

21578

In [10]:
VAL_START

14631

In [11]:
hard_dataset[0]['passage']

'Milhouse Mussolini van Houten is a fictional character featured in the animated television series "The Simpsons", voiced by Pamela Hayden, and created by Matt Groening who named the character after President Richard Nixon\'s middle name. Later in the series, it is revealed that Milhouse\'s middle name is "Mussolini." #$ Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician. [...] .  Her videos are posted on YouTube under the name of Cossbysweater. Subjects of her songs have included the film "The Room", the character Milhouse from the television show "The Simpsons", and the game Dungeons & Dragons. Her style has been compared to that of Bo Burnham. In December 2015, Goertz released a concept album based on the Adult Swim series "Rick and Morty", "Sad Dance Songs", with the album\'s cover emulating the animation and logo of the series. The album was made possible through Kickstarter. She is co-host of Everything\'s Coming Up Podcast, a Simpsons-focused podcast along 

## Utils

In [12]:

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)



def evaluate_truth_pred(truths, preds):
    '''
    truths, preds: matched arrays of ground truth answers and predictions
    '''
    f1 = exact_match = total = 0
    for i in range(len(truths)):
        total += 1
        if truths[i] in ['yes', "no"]:
            continue
        ground_truths = [truths[i]]
        prediction = preds[i]
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / (total+ 0.00000000001)
    f1 = 100.0 * f1 / (total+ 0.00000000001)

    return {'exact_match': exact_match, 'f1': f1}


def get_answer( model, tokenizer, context, question):
    # 1. TOKENIZE THE INPUT
    # note: if you don't include return_tensors='pt' you'll get a list of lists which is easier for
    # exploration but you cannot feed that into a model.
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    inputs = inputs.to(device)
    # 2. OBTAIN MODEL SCORES
    # the AutoModelForQuestionAnswering class includes a span predictor on top of the model.
    # the model returns answer start and end scores for each word in the text
    answer_start_scores, answer_end_scores = model(**inputs, return_dict=False)
    answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
    # 3. GET THE ANSWER SPAN
    # once we have the most likely start and end tokens, we grab all the tokens between them
    # and convert tokens back to words!
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer
# https://huggingface.co/transformers/migration.html
def model_evaluation(model, tokenizer, questions, contexts, answers, to_print = False):
    preds = []
    my_list = list(range(len(questions)))
    with tqdm(total=len(my_list)) as pbar:
        for ex in range(len(questions)):
            answer = get_answer(model, tokenizer, contexts[ex],questions[ex])
            preds.append(answer)
            if ex % 100 == 0 and to_print:
                print("context " +contexts[ex] )
                print("quest " +questions[ex] )
                print("truth " +answers[ex]['text'] )
                print("pred " + answer)
            pbar.update(1)
    truths = [answers[i]['text'] for i in range(len(answers))]
    scores = evaluate_truth_pred(truths, preds)
    print(scores)
    return scores




In [13]:
from sklearn.metrics.pairwise import rbf_kernel
def kernel_similarity(x, y):
        kernel_dist = rbf_kernel(x.reshape(1, -1),y.reshape(1, -1))
        return kernel_dist[0][0]


# FakeAI model

In [14]:
class FakeAI:
    r"""
    FakeAI that is incorrect in random clusters in the input space
    
    Args:
        train_passages: list of embedded passages
        train_questions: list of embedded questions
        n_clus_p: number of centroids in kmeans for passages
        n_clus_q: number of centroids in kmeans for questions
        clust_err_p: clusters where AI makes error on passages
        clust_err_q: clusters where AI makes error on questions
    """
    def __init__(self, train_passages, train_questions, n_clus_p, n_clus_q, n_clust_err_p, n_clust_err_q):
 
        self.train_passages = train_passages
        self.train_questions = train_questions
        self.n_clus_p = n_clus_p
        self.n_clus_q = n_clus_q
        self.n_clust_err_p = n_clust_err_p
        self.n_clust_err_q = n_clust_err_q
        self.build_kmeans()
        self.clust_err_p = random.sample(list(range(self.n_clus_p)), self.n_clust_err_p)
        self.clust_err_q = random.sample(list(range(self.n_clus_q)), self.n_clust_err_q)
        
        
    def build_kmeans(self):
        # Builds kmeans for passages and questions
        self.kmeans_quest = KMeans(n_clusters=self.n_clus_q, max_iter = 10000).fit(self.train_questions)
        self.kmeans_pass = KMeans(n_clusters=self.n_clus_p, max_iter = 100000).fit(self.train_passages)

    def predict_right_wrong(self, passages, questions):
        '''
        Args:
            passages: list of embedded passages
            questions: list of embedded questions (same size as passages)
        Returns:
            preds: binary array indicating if AI is right (1) or wrong (0)
        '''
        clusts_p = self.kmeans_pass.predict(passages)
        clusts_q = self.kmeans_quest.predict(questions)
        preds = []
        for i in range(len(clusts_p)):
            if (clusts_p[i] in self.clust_err_p) or (clusts_q[i] in self.clust_err_q):
                preds.append(0)
            else:
                preds.append(1)
        return preds

    def predict_proba(self, raw_passages):
        embed_ps = model.encode(raw_passages)
        embed_ps = [embed_ps[i] for i in range(len(embed_ps))]
        clusts_p = self.kmeans_pass.predict(embed_ps)
        preds = []
        for i in range(len(clusts_p)):
            if (clusts_p[i] in self.clust_err_p):
                preds.append([0.0,1.0])
            else:
                preds.append([1.0,0.0])
        return np.asarray(preds)


def is_a_stopword(feature, weight):
    split_words = feature.split(' ')
    for word in split_words:
        if word in stopwords:
            return False
    return True

def get_highlighted_p(ai_model, paragraph):
    '''
    from https://eli5.readthedocs.io/en/latest/tutorials/black-box-text-classifiers.html#lime-tutorial
    '''
    
    te = TextExplainer(random_state=42)
    te.fit(paragraph, ai_model.predict_proba)
    show_pred = te.show_prediction(target_names=['correct','false'], top=(8,0), feature_filter = is_a_stopword)
    correct = False
    if show_pred.data.find("y=correct") != -1:
        correct = True
    b = show_pred.data.split('<p style="margin-bottom: 2.5em; margin-top:-0.5em;">')
    to_show = '<p style="margin-bottom: 2.5em; margin-top:-0.5em;">' +b[-1]
    if not correct:
        to_show = to_show.replace("hsl(120,","hsl(0,")
    return to_show

In [15]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v1') #distilbert-base-nli-stsb-mean-tokens
# https://www.sbert.net/docs/pretrained_models.html

# Check clusters 

In [16]:
all_passages = [hard_dataset[i]['passage'] for i in range(VAL_START,len(dataset_indices))]
embeddings_passage = model.encode(all_passages)

all_intros = [hard_dataset[i]['intros'] for i in range(VAL_START,len(dataset_indices))]
embeddings_intro = model.encode(all_intros)

all_questions = [hard_dataset[i]['question'] for i in range(VAL_START,len(dataset_indices))]
embeddings_question = model.encode(all_questions)

all_answers = [hard_dataset[i]['answer'] for i in range(VAL_START,len(dataset_indices))]
embeddings_ans= model.encode(all_answers)

In [19]:
N_CLUSTERS = 15
kmeans_pass = KMeans(n_clusters=15,  max_iter = 10000, random_state = 66).fit(embeddings_passage)


Understand the topic of clusters from wikipedia categories of articles

In [22]:
all_categories = []
def is_valid(title):
    not_allowed = ["Wikipedia","Wikidata", "articles","Articles","Year of birth","Living people"]
    for term in not_allowed:
        if term in title:
            return False
    return True
        
for i in range(0, len(all_passages)):
    if i % 50 == 0:
        print(i)
    #result_search = search(all_passages[i][0:100], tld='com', lang='en', num=2, start=0, stop=1, pause=1.0)
    #search_result = next(result_search)
    #if "wiki" not in search_result:
    article_passage = wikipedia.search(all_passages[i][0:30])
    if len(article_passage) == 0:
        article_passage = wikipedia.search(all_passages[i][0:20])
        if len(article_passage) == 0:
            article_passage = wikipedia.search(all_passages[i][0:10])
            if len(article_passage) == 0:
                article_passage = wikipedia.search(all_passages[i][0:5])
                if len(article_passage) == 0:
                    print("couldnt find anything, stopped trying")
                    all_categories.append([])
                    continue
                else:
                    article_passage = article_passage[0]
            else:
                article_passage = article_passage[0]
        else:
            article_passage = article_passage[0]

    else:
        article_passage = article_passage[0]
    '''
    else:
       article_passage = wikipedia.search(search_result.split('/wiki/')[1])
        if len(article_passage) == 0:
            all_categories.append([])
            continue
        else:
            article_passage = article_passage[0]
    '''
    try:
        wiki_page = wikipedia.WikipediaPage(article_passage)
        categs = wiki_page.categories
        categs = [categs[i] for i in range(len(categs)) if is_valid(categs[i])]
        all_categories.append(categs)
    except:
        print("something bad happened")
        all_categories.append([])

0


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


something bad happened
something bad happened
50
100
150
200
something bad happened
250
something bad happened
300
something bad happened
something bad happened
350
something bad happened
400
couldnt find anything, stopped trying
450
something bad happened
500
550
600
650
700
something bad happened
750
800
850
something bad happened
900
950
1000
1050
1100
something bad happened
1150
couldnt find anything, stopped trying
1200
1250
1300
1350
something bad happened
1400
1450
something bad happened
1500
1550
1600
something bad happened
something bad happened
something bad happened
1650
1700
1750
something bad happened
something bad happened
1800
1850
1900
1950
2000
something bad happened
2050
something bad happened
2100
something bad happened
2150
something bad happened
2200
2250
2300
2350
2400
something bad happened
2450
2500
something bad happened
2550
2600
something bad happened
2650
2700
something bad happened
2750
something bad happened
something bad happened
2800
2850
something bad h

In [26]:
from sklearn_extra.cluster import KMedoids

passages = [[] for _ in range(N_CLUSTERS)]
embeddings = [[] for _ in range(N_CLUSTERS)]
categories = [[] for _ in range(N_CLUSTERS)]
for i in range(len(all_categories)):
    cluster_index = kmeans_pass.labels_[i]
    passages[cluster_index].append(all_passages[i])
    embeddings[cluster_index].append(embeddings_passage[i])
    for categ in all_categories[i]:
        categories[cluster_index].append(categ)
    #hard_dataset
    #if (cluster_index == 0): #and (i%2 == 0):
        #print("###############")
        #print(validation_dataset[i]['answer'])
        #print(hard_dataset[i]['passage'])
        #print(validation_dataset[i]['question'])

              #      print(validation_dataset[i]['answer'])
from collections import Counter


for i in range(N_CLUSTERS):
    print("########################################")
    print("############## CLUSTER "+str(i)+"##############")
    print("########################################")
    #if i not in [5]:
    #    continue
    
    c = Counter(categories[i])
    mc = c.most_common(30)
    for tt in mc:
        if "CS1" not in tt[0] and "Pages" not in tt[0] and "Webarchive" not in tt[0] :
            print(tt[0])
    #print(c.most_common(20))

    print("\n")

########################################
############## CLUSTER 0##############
########################################
English-language television shows
21st-century American actresses
American television actresses
American film actresses
AC with 0 elements
American voice actresses
20th-century American actresses
American male television actors
American male film actors
20th-century American male actors
21st-century American male actors
American people of English descent
American films
English-language films
21st-century American comedians


########################################
############## CLUSTER 1##############
########################################
Taxonbars with 20–24 taxon IDs
Taxa named by Carl Linnaeus
FCI breeds
Harv and Sfn no-target errors
Taxonbars with 25–29 taxon IDs
IUCN Red List least concern species
AC with 0 elements
Herding dogs
Garden plants
Poaceae genera


########################################
############## CLUSTER 2##############
###################